In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellem.musique.qa import answer_question_standard, answer_question_cot, answer_question_cot_fs, answer_question_cte
from bellem.utils import set_seed, jprint
from bellem.musique.singlehop import benchmark

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
pd.options.display.float_format = '{:,.3f}'.format

In [5]:
def perfect_retrieval_func(docs, query):
    return [doc for doc in docs if doc['is_supporting']]

In [6]:
N_RUNS = 3

In [7]:
from bellem.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-common/base-dataset-validation.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
# df = df.sample(10)

print(df.shape)
df.head()

(100, 8)


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [8]:
results = []

for qa_technique, qa_func in tqdm(
    [
        ("standard", answer_question_standard),
        ("cot-zs", answer_question_cot),
        ("cot-fs", answer_question_cot_fs),
        ("cte", answer_question_cte),
    ]
):
    for run in range(1, N_RUNS + 1):
        _, scores = benchmark(df, qa_func, perfect_retrieval_func, ignore_errors=False)
        results.append(
            {
                **scores,
                "retrieval": "groundtruth",
                "context": "paragraphs",
                "qa": qa_technique,
                "run": run,
            }
        )

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Report

In [ ]:
report_df = pd.DataFrame.from_records(results, columns=['context', 'retrieval', 'qa', 'run', 'exact_match', 'f1'])
report_df

In [ ]:
from datetime import datetime
suffix = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
report_df.to_json(f'./ablation-prompting-technique-{suffix}.jsonl', orient='records', lines=True)

In [ ]:
report_df.drop(columns=['context', 'retrieval', 'run']).groupby(['qa']).agg(['min', 'mean', 'max', 'std'])

## Inspect

In [ ]:
fail_mask = ~(df_cte['fuzzy_match'])

In [ ]:
df_cte.loc[fail_mask]['predicted_answer']

In [ ]:
run = 3
row = df_cte.loc[fail_mask].iloc[run]

print("="*80)
print(row['question'])
print(row['answers'])

print("="*80)
jprint(row['raw_output'])